In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import clear_output
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import optimisation as op

In [ ]:
# make deterministic
np.random.seed(100)

Function to optimize:

In [ ]:
f = lambda x: x * np.cos(x)
x_min = 0
x_max = 12
xs = np.linspace(x_min, x_max, num=200)
ys = f(xs)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(xs, ys, 'g-')
plt.margins(0.1, 0.1)
plt.xlabel('x')
plt.ylabel('cost')
plt.show()

In [ ]:
ranges = {
    'x': xs,
    'y': [5],
    #'z': np.linspace(0, 10, num=10)
}
gp_params = dict(
    alpha = 1e-5, # larger => more noise. Default = 1e-10
    kernel = gp.kernels.Matern(length_scale=1),
    n_restarts_optimizer = 10,
    # make the mean 0 (theoretically a bad thing, see docs, but can help)
    normalize_y = True,
    copy_X_train = True # make a copy of the training data
)

In [ ]:
class MyEvaluator(op.LocalEvaluator):
    def test_config(self, config):
        return f(config.x)

In [ ]:
optimiser = op.BayesianOptimisationOptimiser(ranges,
                                            acquisition_function='EI',
                                            gp_params=None,
                                            max_steps=4,
                                            pre_samples=3,
                                            ac_num_restarts=10)
optimiser.step_log_keep = 100
evaluator = MyEvaluator(optimiser)

In [ ]:
optimiser.run()
evaluator.start()

In [ ]:
evaluator.wait_for()
optimiser.wait_for()

In [ ]:
print(evaluator.log_record)

In [ ]:
print(optimiser.logger)

In [ ]:
optimiser.max_steps += 5
optimiser.run()
evaluator.start()
evaluator.wait_for()
optimiser.wait_for()

In [ ]:
optimiser.step_log.keys()

In [ ]:
for i in sorted(optimiser.step_log.keys()):
    clear_output(wait=True)
    fig = optimiser.plot_step_slice('x', i, ys, True)
    #fig = optimiser.plot_step_slice('z', i)
    #fig.savefig('frames/{:02}.png'.format(i))
    time.sleep(0.5)